In [1]:
import tensorflow as tf
import numpy as np
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
tf.__version__

'1.12.0'

In [3]:
x_train = np.load('./data/mnist/x_train.npy')
x_test = np.load('./data/mnist/x_test.npy')
y_train_cls = np.load('./data/mnist/y_train.npy')
y_test_cls = np.load('./data/mnist/y_test.npy')

In [4]:
num_pixels = x_train.shape[1]
num_channels = 1

In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [6]:
x_train = x_train/255
x_test = x_test/255

In [7]:
y_train = np_utils.to_categorical(y_train_cls)
y_test = np_utils.to_categorical(y_test_cls)
num_classes = y_train[0].size

In [8]:
filter_size1 = 5
num_filters1 = 16

filter_size2 = 5
num_filters2 = 36

fc_size = 128

In [9]:
x = tf.placeholder(tf.float32, [None, num_pixels, num_pixels])
x_img = tf.reshape(x,[-1,num_pixels,num_pixels,num_channels])

In [10]:
y_true = tf.placeholder(tf.float32, [None, num_classes])
y_true_cls = tf.placeholder(tf.int64, [None])

In [11]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

In [12]:
def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))

In [13]:
def new_conv_layer(input, num_input_channels, filter_size, num_filters, use_pooling=True):
    shape = [filter_size, filter_size, num_input_channels, num_filters]
    weights = new_weights(shape=shape)
    biases = new_biases(length=num_filters)
    
    layer = tf.nn.conv2d(input = input, filter = weights, strides = [1,1,1,1], padding='SAME')
    layer += biases
    
    if use_pooling:
        layer = tf.nn.max_pool(value=layer, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    layer = tf.nn.relu(layer)
    print("No. of parameters: " +  str(layer.get_shape()[1:4].num_elements()))
    return layer, weights

In [14]:
def flatten_layer(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer_flat = tf.reshape(layer, [-1,num_features])
    return layer_flat, num_features

In [15]:
def new_fc_layer(input,num_inputs, num_outputs, use_relu=True):
    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)
    layer=tf.matmul(input,weights)+biases
    if use_relu:
        layer = tf.nn.relu(layer)
    return layer

In [16]:
layer_conv1, weights_conv1 = new_conv_layer(input=x_img, num_input_channels=num_channels, filter_size=filter_size1, num_filters=num_filters1, use_pooling=True)

No. of parameters: 3136


In [17]:
layer_conv1

<tf.Tensor 'Relu:0' shape=(?, 14, 14, 16) dtype=float32>

In [18]:
layer_conv2, weights_conv2 = new_conv_layer(input=layer_conv1, num_input_channels=num_filters1, filter_size=filter_size2, num_filters=num_filters2, use_pooling=True)

No. of parameters: 1764


In [19]:
layer_conv2

<tf.Tensor 'Relu_1:0' shape=(?, 7, 7, 36) dtype=float32>

In [20]:
layer_flat, num_features = flatten_layer(layer_conv2)

In [21]:
layer_flat

<tf.Tensor 'Reshape_1:0' shape=(?, 1764) dtype=float32>

In [22]:
layer_fc1 = new_fc_layer(input=layer_flat, num_inputs=num_features, num_outputs=fc_size,use_relu=True)

In [23]:
layer_fc2 = new_fc_layer(input=layer_fc1, num_inputs=fc_size, num_outputs=num_classes,use_relu=False)

In [24]:
layer_fc2

<tf.Tensor 'add_3:0' shape=(?, 10) dtype=float32>

In [25]:
y_pred = tf.nn.softmax(layer_fc2)
y_pred_cls = tf.argmax(y_pred, axis=1)

In [26]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=layer_fc2, labels=y_true)

In [27]:
cost = tf.reduce_mean(cross_entropy)

In [28]:
lr = 0.0001
optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(cost)

In [29]:
correct_prediction = tf.equal(y_pred_cls, y_true_cls)

In [30]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [31]:
session=tf.Session()

In [32]:
session.run(tf.global_variables_initializer())

In [ ]:
def print_accuracy():
    acc = session.run(accuracy, feed_dict = feed_dict_test)
    print("Accuracy on test-set:{0:.1%}".format(acc))

In [33]:
batch_size = 64

In [34]:
def optimize(num_iterations):
    for j in range(num_iterations):
        print("Iteration: " + str(j))
        for i in range(batch_size, x_train.shape[0], batch_size):
            x_batch = x_train[i-batch_size:i]
            y_true_batch = y_train[i-batch_size:i]
            feed_dict_train = {x: x_batch, y_true: y_true_batch}
            session.run(optimizer, feed_dict=feed_dict_train)
        print_accuracy()

In [35]:
feed_dict_test = {x:x_test, y_true: y_test, y_true_cls: y_test_cls}

In [37]:
print_accuracy()

In [38]:
weights_conv1.eval(session=session)

array([[[[-1.83622893e-02, -1.95531175e-02,  2.69870758e-02,
          -1.09492894e-02, -5.39430883e-03, -5.08253044e-03,
          -2.57888623e-02, -1.96058564e-02,  1.90910436e-02,
          -8.34563822e-02,  3.17862560e-03,  4.68894951e-02,
          -7.63530061e-02, -4.07735184e-02,  5.02007864e-02,
           5.30847311e-02]],

        [[ 4.22851257e-02, -8.92702565e-02, -4.53758193e-03,
           2.93086153e-02,  1.43771889e-02, -3.68995010e-03,
           5.51007781e-03,  2.16007489e-03,  4.01154533e-02,
           2.41086911e-02,  6.11188225e-02, -6.88165575e-02,
           5.18317756e-05,  5.30268066e-02,  1.00708026e-02,
          -2.96863019e-02]],

        [[-3.69109064e-02,  1.68617144e-02,  4.39846963e-02,
          -1.46965189e-02, -6.29031137e-02, -8.05874690e-02,
          -6.33371174e-02, -5.19741252e-02, -4.70990241e-02,
          -2.76160869e-03, -3.17893587e-02, -1.25446767e-02,
           6.50916100e-02,  1.34970723e-02, -1.94671527e-02,
          -4.09251489e-02

In [ ]:
weights_conv2.eval(session=session)

In [39]:
optimize(num_iterations=2)

Iteration: 0
